In [1]:
!pip install kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"cum65705","key":"959cd786169c99d59f9c216c4551b71e"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d snap/amazon-fine-food-reviews

 98% 237M/242M [00:01<00:00, 163MB/s]
100% 242M/242M [00:01<00:00, 144MB/s]


In [4]:
!unzip amazon-fine-food-reviews.zip 
!ls

Archive:  amazon-fine-food-reviews.zip
  inflating: Reviews.csv             
  inflating: database.sqlite         
  inflating: hashes.txt              
amazon-fine-food-reviews.zip  hashes.txt   Reviews.csv
database.sqlite		      kaggle.json  sample_data


In [1]:
!pip install pyLDAvis

In [2]:
import pandas as pd
import numpy as np

import re
import string

import spacy

import gensim
from gensim import corpora

# libraries for visualization
import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [3]:
review_data= pd.read_csv("./Reviews.csv")
print(review_data.head(2))
print(len(review_data))
print('Unique Products')
print(len(review_data.groupby('ProductId')))
print('Unique Users')
print(len(review_data.groupby('UserId')))

   Id  ...                                               Text
0   1  ...  I have bought several of the Vitality canned d...
1   2  ...  Product arrived labeled as Jumbo Salted Peanut...

[2 rows x 10 columns]
568454
Unique Products
74258
Unique Users
256059


In [4]:
def clean_text(text ): 
    delete_dict = {sp_character: '' for sp_character in string.punctuation} 
    delete_dict[' '] = ' ' 
    table = str.maketrans(delete_dict)
    text1 = text.translate(table)
    #print('cleaned:'+text1)
    textArr= text1.split()
    text2 = ' '.join([w for w in textArr if ( not w.isdigit() and  ( not w.isdigit() and len(w)>3))]) 
    
    return text2.lower()

In [5]:
import nltk
nltk.download('stopwords') # run this one time

/usr/local/lib/python3.7/dist-packages/nltk/decorators.py:70: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  formatvalue=lambda value: "")[1:-1]
/usr/local/lib/python3.7/dist-packages/scipy/io/matlab/mio5.py:98: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  from .mio5_utils import VarReader5


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
review_data.dropna(axis = 0, how ='any',inplace=True) 






review_data['Text'] = review_data['Text'].apply(clean_text)
review_data['Num_words_text'] = review_data['Text'].apply(lambda x:len(str(x).split())) 

print('-------Dataset --------')
print(review_data['Score'].value_counts())
print(len(review_data))
print('-------------------------')
max_review_data_sentence_length  = review_data['Num_words_text'].max()

mask = (review_data['Num_words_text'] < 100) & (review_data['Num_words_text'] >=20)
df_short_reviews = review_data[mask]
df_sampled = df_short_reviews.groupby('Score').apply(lambda x: x.sample(n=20000)).reset_index(drop = True)

print('No of Short reviews')
print(len(df_short_reviews))



#all_sentences = train_data['text'].tolist() + test_data['text'].tolist()

-------Dataset --------
5    363111
4     80655
1     52264
3     42638
2     29743
Name: Score, dtype: int64
568411
-------------------------
No of Short reviews
373281


In [7]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
# function to remove stopwords
def remove_stopwords(text):
    textArr = text.split(' ')
    rem_text = " ".join([i for i in textArr if i not in stop_words])
    return rem_text

# remove stopwords from the text
df_sampled['Text']=df_sampled['Text'].apply(remove_stopwords)

In [8]:
!pip install spacy

In [9]:
!python -m spacy download en_core_web_md

     |████████████████████████████████| 96.4 MB 1.5 MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.5-py3-none-any.whl size=98051302 sha256=25d7cd61a884afd12e229bd76b75190b87eb1b81326dc933c297016362af0677
  Stored in directory: /tmp/pip-ephem-wheel-cache-kvcxuimq/wheels/69/c5/b8/4f1c029d89238734311b3269762ab2ee325a42da2ce8edb997
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [16]:
import en_core_web_md
nlp = en_core_web_md.load( disable=['parser', 'ner'])

def lemmatization(texts,allowed_postags=['NOUN', 'ADJ']): 
       output = []
       for sent in texts:
             doc = nlp(sent) 
             output.append([token.lemma_ for token in doc if token.pos_ in allowed_postags ])
       return output

/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:126: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):


In [17]:
text_list=df_sampled['Text'].tolist()
print(text_list[1])
tokenized_reviews = lemmatization(text_list)
print(tokenized_reviews[1])

never left review hopes help someone else item comes typing cups believe machine uses cups spent money something need throw away single serving flat round filter product rating fact amazon searchable something clearly


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups di

['review', 'hope', 'else', 'item', 'cup', 'machine', 'cup', 'money', 'single', 'flat', 'round', 'filter', 'product', 'rating', 'fact']


/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use sele

In [18]:
dictionary = corpora.Dictionary(tokenized_reviews)
doc_term_matrix = [dictionary.doc2bow(rev) for rev in tokenized_reviews]

In [19]:
# Creating the object for LDA model using gensim library
LDA = gensim.models.ldamodel.LdaModel

# Build LDA model
lda_model = LDA(corpus=doc_term_matrix, id2word=dictionary, num_topics=10, random_state=100,
                chunksize=1000, passes=50,iterations=100)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(El

In [20]:
lda_model.print_topics()

[(0,
  '0.102*"food" + 0.033*"chip" + 0.026*"salt" + 0.020*"cat" + 0.019*"good" + 0.014*"healthy" + 0.013*"baby" + 0.013*"brand" + 0.012*"chicken" + 0.012*"year"'),
 (1,
  '0.037*"cookie" + 0.030*"good" + 0.028*"product" + 0.027*"sugar" + 0.025*"sauce" + 0.019*"protein" + 0.017*"ingredient" + 0.016*"great" + 0.016*"calorie" + 0.015*"rice"'),
 (2,
  '0.038*"product" + 0.037*"time" + 0.027*"order" + 0.021*"year" + 0.019*"great" + 0.014*"happy" + 0.014*"month" + 0.013*"good" + 0.012*"gift" + 0.011*"fresh"'),
 (3,
  '0.092*"chocolate" + 0.054*"milk" + 0.037*"bar" + 0.027*"water" + 0.026*"coconut" + 0.019*"delicious" + 0.017*"cake" + 0.015*"fiber" + 0.015*"dark" + 0.013*"value"'),
 (4,
  '0.045*"butter" + 0.042*"peanut" + 0.039*"candy" + 0.028*"cheese" + 0.023*"energy" + 0.021*"cracker" + 0.019*"delivery" + 0.018*"hair" + 0.015*"bottle" + 0.012*"olive"'),
 (5,
  '0.052*"snack" + 0.036*"flavor" + 0.031*"good" + 0.027*"sweet" + 0.023*"fruit" + 0.023*"cereal" + 0.022*"great" + 0.022*"healthy" 

In [22]:
# Visualize the topics
#https://github.com/bmabey/pyLDAvis
#https://speakerdeck.com/bmabey/visualizing-topic-models
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, doc_term_matrix, dictionary)
vis

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8     -0.011856  0.188545       1        1  14.320946
6      0.135842 -0.101431       2        1  13.990600
9      0.152309 -0.053191       3        1  11.589850
2      0.202635 -0.119522       4        1  11.324132
0      0.136975 -0.025589       5        1  10.935465
7     -0.075064  0.191711       6        1   9.504073
1      0.058676  0.066028       7        1   9.087849
5     -0.073148  0.186920       8        1   8.157111
3     -0.303874 -0.022540       9        1   5.848952
4     -0.222494 -0.310929      10        1   5.241022, topic_info=          Term          Freq         Total Category  logprob  loglift
2       coffee  28376.000000  28376.000000  Default  30.0000  30.0000
125       food  20885.000000  20885.000000  Default  29.0000  29.0000
278     flavor  32183.000000  32183.000000  Default  28.0000  28.0000
77       price  13523.000000  13523.000000  Default  27.0000  27.0000
198  chocolate  10079.000000  10079.000000  Default  26.0000  26.0000
..         ...           ...           ...      ...      ...      ...
142        nut    516.941065    936.244856  Topic10  -5.2415   2.3547
293    plastic    566.823023   1435.964840  Topic10  -5.1494   2.0191
28     product   1054.827224  27191.211343  Topic10  -4.5283  -0.3009
844      glass    438.045463   1277.929845  Topic10  -5.4071   1.8780
361       nice    441.245336   7796.314890  Topic10  -5.3999   0.0768

[544 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
369       2  0.310008      able
369       4  0.538326      able
369       5  0.070068      able
369       7  0.081347      able
3076      4  0.998065  additive
...     ...       ...       ...
117       5  0.214809      year
117       6  0.026695      year
117       7  0.058730      year
287       3  0.998622    yogurt
1842      8  0.999513     yummy

[781 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 7, 10, 3, 1, 8, 2, 6, 4, 5])